# 1. kitti 2 yolov5
- kitti의 픽셀 포멧의 라벨링을 yolov5에 맞게 비율로 변환해주었습니다.
- kitti는 vkitti와 다르게 (left, right, top, bottom)이 아니라(left, top, right, bottom)의 형식으로 라벨링되어 있습니다.
- 라벨링을 해줄 때 관리하기 쉽게 인덱싱해주었습니다.<br>
    ex) 키티 : 0000(scene번호) + 000000(이미지 프레임).txt

In [9]:
import pandas as pd
import os
import shutil

IMAGE_SIZE = (1242, 375)

In [10]:
def convert(bbox_path,save_path):
    #파일 불러오기
    df = pd.read_csv(bbox_path, sep=" ", header=None)
    df = df.drop([1,3,4,5,10,11, 12, 13, 14,15,16], axis=1)

    # vkitti: l r t b : 3,4,5,6
    # kitti: l t r b : 6,7,8,9
    #kitti2yolo format 변환
    df['x'] = ((df[6].astype(int) + df[8].astype(int))/2.0) * (1./IMAGE_SIZE[0])
    df['y'] = ((df[7].astype(int) + df[9].astype(int))/2.0) * (1./IMAGE_SIZE[1])
    df['w'] = (df[8].astype(int) - df[6].astype(int)) * (1./IMAGE_SIZE[0])
    df['h'] = (df[9].astype(int) - df[7].astype(int)) * (1./IMAGE_SIZE[1])
    df = df.drop([6, 7, 8, 9], axis=1)
    
    #classNo 변환 (Car : 0, Van : 1, Truck : 2)
    df.loc[df[2] == 'Car','classNo'] = '0'
    df.loc[df[2] == 'Van','classNo'] = '1'
    df.loc[df[2] == 'Truck','classNo'] = '2'

    df[2] = df['classNo']
    df = df.drop(['classNo'], axis=1)
    df = df.dropna(axis=0)

    frames = []
    for i in range(int(df.loc[df.tail(1).index][0])+1):
        tmp = df[df[0] == i]
        tmp = tmp.drop([0],axis = 1)
        frames.append(tmp)
    
    # df 저장 scene[4] + frame[6]
    for i, frame in enumerate(frames):
        index = str(i).zfill(6)
        frame.to_csv('{}{}{}.txt'.format(save_path,os.path.basename(bbox_path)[:4],index), 
                    sep=' ', header=False, index=False)

In [14]:
data_path = '/usr/src/datasets/raw_kitti/raw_kitti_bbox/'
save_bbox_path = '/usr/src/datasets/kitti2015/labels/'

if os.path.exists(save_bbox_path):
    shutil.rmtree(save_bbox_path)
os.mkdir(save_bbox_path)

sList = ['0001.txt','0002.txt','0006.txt','0018.txt','0020.txt']
for scene in sList:
    convert(data_path+scene,save_bbox_path)
    print('scene '+scene[:4]+' converted to yolov5 format')

scene 0001 converted to yolov5 format
scene 0002 converted to yolov5 format
scene 0006 converted to yolov5 format
scene 0018 converted to yolov5 format
scene 0020 converted to yolov5 format
